In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage,  AIMessage



OPENAI_KEY = os.getenv("OPENAI_API_KEY") 

In [2]:
chat = ChatOpenAI(
    openai_api_key=OPENAI_KEY,
    model='gpt-3.5-turbo'
)



messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

res = chat(messages)
res

/home/melat/tnx/week6/Enterprise-Grade-RAG-Systems/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/melat/tnx/week6/Enterprise-Grade-RAG-Systems/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Sure! String theory is a theoretical framework in physics that aims to provide a unified description of all fundamental particles and forces in the universe. It suggests that the fundamental building blocks of the universe are not point-like particles, but tiny, vibrating strings.\n\nHere are a few key points to understand about string theory:\n\n1. Dimensions: String theory requires the existence of extra dimensions beyond the three spatial dimensions (length, width, and height) we are familiar with. In fact, most versions of string theory propose that there are a total of ten dimensions, with six of them "curled up" into tiny, compact shapes.\n\n2. Vibrating Strings: In string theory, particles are not considered as separate entities, but rather as different modes of vibrations of these tiny strings. The different vibrational patterns of the strings correspond to different types of particles and their properties, such as mass and charge.\n\n3. Quantum Mechanics and

Load our context to the chroma vector database

In [7]:
import os
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
def create_qa_model():
    # Load documents
    loader = PyPDFLoader("./data/RAG.pdf")
    documents = loader.load()
    # Split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    print(len(texts))
    # Select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # Create the vector store to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # Expose this index in a retriever interface
    retriever = db.as_retriever( search_kwargs={"k": 3})
    return  retriever

In [4]:
create_qa_model()

11


VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fcebd043a60>, search_kwargs={'k': 2})

setup QA chain

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

retriver = create_qa_model()
primary_qa = ChatOpenAI(model_name='gpt-3.5-turbo-16k' )
qa_chain = RetrievalQA.from_chain_type(primary_qa,retriever = retriver, return_source_documets= True)

11


ValidationError: 1 validation error for RetrievalQA
return_source_documets
  extra fields not permitted (type=value_error.extra)